In [2]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt

#tf.set_random_seed(1)

%run Dataset.ipynb
%run ImageGenerator.ipynb

IMAGE_HEIGHT = 100
IMAGE_WEIGHT = 100

BATCH_SIZE = 32


data_dir = os.getcwd()
X_path = data_dir + '/Ze'
Y_path = data_dir + '/Ze' #""#autoencoder##data_dir + '/y'

image_generator = ImageGenerator(IMAGE_HEIGHT, IMAGE_WEIGHT)
data_set = Dataset(X_path, Y_path, BATCH_SIZE, image_generator)

#CHECK
data_set.current_batch_index = 3
check_images, _ = np.array(data_set.get_next_batch(32))
check_image = check_images[4]

print(check_images.shape)
plt.imshow(check_image)

(32, 100, 100, 3)


In [3]:
from tensorflow.layers import conv2d, max_pooling2d, conv2d_transpose

tf.reset_default_graph() 

X = tf.placeholder(dtype=tf.float32, shape=(None, IMAGE_HEIGHT, IMAGE_WEIGHT, 3))

x = conv2d(X, filters=64, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', name = 'conv1')
x = max_pooling2d(x, pool_size=(2, 2), strides=(2, 2), padding='SAME', name='pool1')
x = conv2d(x, filters=32, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', name='conv2')
x = max_pooling2d(x, pool_size=(2, 2), strides=(2, 2), padding='SAME', name='pool2')

x = conv2d(x, filters=32, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', name='conv3')
x = max_pooling2d(x, pool_size=(2, 2), strides=(2, 2), padding='SAME', name='pool2')

x = conv2d(x, filters=24, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', name='conv4')
encoded = max_pooling2d(x, pool_size=(2, 2), strides=(2, 2), padding='SAME', name='encoded')

# Decoder
x = conv2d(encoded, filters=24, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', name='deconv1')
x = conv2d_transpose(x, filters=16, kernel_size=(2, 2), strides=(2, 2), name='conv2d_transpose1')

x = conv2d(x, filters=32, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', name='deconv2')
x = conv2d_transpose(x, filters=16, kernel_size=(2, 2), strides=(2, 2), name='s_deconv2')

x = conv2d(x, filters=32, kernel_size=(3, 3), activation=tf.nn.relu, name='s_conv3')
x = conv2d_transpose(x, filters=16, kernel_size=(2, 2), strides=(2, 2), name='s_deconv3')


x = conv2d(x, filters=64, kernel_size=(3, 3), activation=tf.nn.relu, name='s_conv4')
x = conv2d_transpose(x, filters=16, kernel_size=(2, 2), strides=(2, 2), name='s_deconv4')

decoded = conv2d(x, filters=3, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', name='recon')

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.conv2d_transpose instead.


In [4]:
import tensorflow.contrib.slim as slim

def model_summary():
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)
    
model_summary()

---------
Variables: name (type shape) [size]
---------
conv1/kernel:0 (float32_ref 3x3x3x64) [1728, bytes: 6912]
conv1/bias:0 (float32_ref 64) [64, bytes: 256]
conv2/kernel:0 (float32_ref 3x3x64x32) [18432, bytes: 73728]
conv2/bias:0 (float32_ref 32) [32, bytes: 128]
conv3/kernel:0 (float32_ref 3x3x32x32) [9216, bytes: 36864]
conv3/bias:0 (float32_ref 32) [32, bytes: 128]
conv4/kernel:0 (float32_ref 3x3x32x24) [6912, bytes: 27648]
conv4/bias:0 (float32_ref 24) [24, bytes: 96]
deconv1/kernel:0 (float32_ref 3x3x24x24) [5184, bytes: 20736]
deconv1/bias:0 (float32_ref 24) [24, bytes: 96]
conv2d_transpose1/kernel:0 (float32_ref 2x2x16x24) [1536, bytes: 6144]
conv2d_transpose1/bias:0 (float32_ref 16) [16, bytes: 64]
deconv2/kernel:0 (float32_ref 3x3x16x32) [4608, bytes: 18432]
deconv2/bias:0 (float32_ref 32) [32, bytes: 128]
s_deconv2/kernel:0 (float32_ref 2x2x16x32) [2048, bytes: 8192]
s_deconv2/bias:0 (float32_ref 16) [16, bytes: 64]
s_conv3/kernel:0 (float32_ref 3x3x16x32) [4608, bytes: 

In [5]:
loss = tf.losses.mean_squared_error(labels=X, predictions=decoded)

learning_rate = 0.0001
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

Instructions for updating:
Use tf.cast instead.


In [6]:
Y = tf.placeholder(dtype=tf.float32, shape=(None, IMAGE_HEIGHT, IMAGE_WEIGHT, 3))
epochs = 10

print_every_ep_index = 5
print_every_b_index = 4


for ep_index in range(epochs):
    #Reset set to 0
    data_set.current_batch_index = 0
    
    if ep_index % print_every_ep_index == 0:
        print('EPOCH #{}'.format(ep_index))
        
    for batch_step in range(data_set.batches_count):
        train_images, _ = np.array(data_set.get_next_batch(BATCH_SIZE))
        batch_loss = sess.run(loss, feed_dict = { X : train_images })
        
        if data_set.current_batch_index % print_every_b_index == 0:
            print('   Loss {}'.format(batch_loss))

EPOCH #0
   Loss 0.35544437170028687
   Loss 0.2928580641746521
   Loss 0.35544437170028687


KeyboardInterrupt: 